In [2]:
!pip install mcp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [mcp]


In [4]:
from mcp.server.fastmcp import FastMCP
import mysql.connector

mcp = FastMCP(name="inventory_mcp")


In [5]:
# MySQL config
db_config = {
    "host": "localhost",
    "user": "root",       # change to your MySQL username
    "password": "",  # change to your MySQL password
    "database": "datascience_schema"
}

In [7]:
import mysql.connector
from mysql.connector import Error

def test_connection(config):
    try:
        connection = mysql.connector.connect(**config)
        if connection.is_connected():
            print("✅ Connection successful!")
            cursor = connection.cursor()
            cursor.execute("SELECT DATABASE(), USER();")
            db_info = cursor.fetchone()
            print(f"Database: {db_info[0]}")
            print(f"User: {db_info[1]}")
            cursor.close()
            connection.close()
            return True
    except Error as e:
        print(f"❌ Connection failed: {e}")
        return False

# Test with your config
test_connection(db_config)

❌ Connection failed: 1045 (28000): Access denied for user 'root'@'localhost' (using password: NO)


False

In [6]:
@mcp.tool()
def add_inventory(item_id: str, product_name: str, location: str, quantity: int) -> dict:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO inventory (item_id, product_name, location, quantity) VALUES (%s, %s, %s, %s) "
        "ON DUPLICATE KEY UPDATE quantity = quantity + %s, product_name = VALUES(product_name)",
        (item_id, product_name, location, quantity, quantity)
    )
    conn.commit()
    conn.close()
    return {"message": f"Added {quantity} units of {product_name} ({item_id}) at {location}"}

